In [2]:
import os 

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ["NVIDIA_VISIBLE_DEVICES"] = '1'

import pandas as pd

In [3]:
df = pd.read_csv('../data/all_data.csv')
df_runews = df[df.dataset_name == 'Kaggle Russian News Dataset'][['sample', 'label']]
df_runews.label = df_runews.label.apply(lambda x: {'negative':0, 'positive':1, 'neutral':1}[x])
df_runews['sample'] = df_runews['sample'].apply(lambda x: x.replace('\n', ' '))
df_runews.reset_index(drop=True, inplace=True)

In [4]:
df_runews.head()

,sample,label
0,Досудебное расследование по факту покупки ЕНПФ...,0
1,Медики рассказали о состоянии пострадавшего му...,0
2,"Прошел почти год, как железнодорожным оператор...",0
3,По итогам 12 месяцев 2016 года на территории р...,0
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,0


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruRoberta-large")
model = AutoModelForSequenceClassification.from_pretrained("sberbank-ai/ruRoberta-large", num_labels=2).cuda()

Some weights of the model checkpoint at sberbank-ai/ruRoberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_pr

In [6]:
from typing import List, Union

import torch
import tqdm
from torch.utils.data import Dataset
from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast


class LabeledDataset(Dataset):
    def __init__(
        self,
        dataframe: pd.DataFrame,
        tokenizer: Union[PreTrainedTokenizerFast, PreTrainedTokenizer],
        max_tokens: int = 512,
        text_field="text",
        res_field="label",
    ):
        self.tokenizer = tokenizer
        self.max_tokens = max_tokens
        self.text_field = text_field
        self.res_field = res_field
        self.dataframe = dataframe

    def embed_record(self, text):
        inputs = self.tokenizer(
            text=text,
            add_special_tokens=True,
            max_length=self.max_tokens,
            padding="max_length",
            truncation="longest_first",
            return_tensors="pt",
        )
        return {key: value.squeeze(0) for key, value in inputs.items()}

    def __len__(self):
        return self.dataframe.shape[0]

    def __getitem__(self, index):
        inputs = self.embed_record(self.dataframe.iloc[index][self.text_field])
        inputs["labels"] = torch.LongTensor(
            [self.dataframe.iloc[index][self.res_field]]
        )
        return inputs


In [7]:
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from transformers import set_seed

set_seed(42)
train_ids, valid_ids = train_test_split(df_runews.index, test_size=0.1, random_state=42)
trainset = LabeledDataset(dataframe = df_runews.iloc[train_ids], tokenizer=tokenizer, text_field='sample', res_field='label', max_tokens=512) 
valset = LabeledDataset(dataframe = df_runews.iloc[valid_ids], tokenizer=tokenizer, text_field='sample', res_field='label', max_tokens=512)

/home/moskovskii/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="../logs/sentiment_experiments/ruroberta_runews_2class_512",
    do_train=True,
    do_eval=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-6,
    weight_decay=1e-6,
    num_train_epochs=10
)

In [9]:
from sklearn.metrics import classification_report
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    f1_score = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print(classification_report(labels, predictions))
    return f1_score.compute(predictions=predictions, references=labels)

In [10]:
from transformers import Trainer 

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=valset,
    compute_metrics=compute_metrics,
)


In [11]:
trainer.train()

/home/moskovskii/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7436
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9300


Epoch,Training Loss,Validation Loss,F1
1,0.351800,0.273760,0.936351
2,0.265700,0.300262,0.938571
3,0.231200,0.379399,0.931365
4,0.176200,0.409592,0.931852
5,0.153700,0.448781,0.942629
6,0.132600,0.527180,0.939971
7,0.131600,0.494255,0.943149
8,0.087100,0.558569,0.946619
9,0.082800,0.568665,0.940833
10,0.063100,0.562223,0.943231


***** Running Evaluation *****
  Num examples = 827
  Batch size = 8


              precision    recall  f1-score   support

           0       0.71      0.56      0.62       134
           1       0.92      0.96      0.94       693

    accuracy                           0.89       827
   macro avg       0.81      0.76      0.78       827
weighted avg       0.88      0.89      0.89       827



Saving model checkpoint to ../logs/sentiment_experiments/ruroberta_runews_2class_512/checkpoint-930
Configuration saved in ../logs/sentiment_experiments/ruroberta_runews_2class_512/checkpoint-930/config.json
Model weights saved in ../logs/sentiment_experiments/ruroberta_runews_2class_512/checkpoint-930/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 827
  Batch size = 8


...

              precision    recall  f1-score   support

           0       0.68      0.76      0.72       134
           1       0.95      0.93      0.94       693

    accuracy                           0.90       827
   macro avg       0.81      0.85      0.83       827
weighted avg       0.91      0.90      0.90       827



Saving model checkpoint to ../logs/sentiment_experiments/ruroberta_runews_2class_512/checkpoint-8370
Configuration saved in ../logs/sentiment_experiments/ruroberta_runews_2class_512/checkpoint-8370/config.json
Model weights saved in ../logs/sentiment_experiments/ruroberta_runews_2class_512/checkpoint-8370/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 827
  Batch size = 8


              precision    recall  f1-score   support

           0       0.69      0.75      0.72       134
           1       0.95      0.94      0.94       693

    accuracy                           0.91       827
   macro avg       0.82      0.84      0.83       827
weighted avg       0.91      0.91      0.91       827



Saving model checkpoint to ../logs/sentiment_experiments/ruroberta_runews_2class_512/checkpoint-9300
Configuration saved in ../logs/sentiment_experiments/ruroberta_runews_2class_512/checkpoint-9300/config.json
Model weights saved in ../logs/sentiment_experiments/ruroberta_runews_2class_512/checkpoint-9300/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9300, training_loss=0.16440595032066427, metrics={'train_runtime': 5454.9307, 'train_samples_per_second': 13.632, 'train_steps_per_second': 1.705, 'total_flos': 6.929841617707008e+16, 'train_loss': 0.16440595032066427, 'epoch': 10.0})